In [32]:
#!/usr/bin/env python

from functools import reduce
import numpy
import scipy.linalg
from pyscf import scf
from pyscf import gto, dft
from pyscf import mcscf, fci
from functools import reduce
import numpy
from pyscf import gto
from pyscf import scf
from pyscf import mcscf
from pyscf import fci
from pyscf.mcscf import avas, dmet_cas


'''
Triplet and quintet energy gap of Iron-Porphyrin molecule

In this example, we use density matrix embedding theory
(ref. Q Sun, JCTC, 10(2014), 3784) to generate initial guess.
'''

#
# For 3d transition metal, people usually consider the so-called double
# d-shell effects for CASSCF calculation.  Double d-shell here refers to 3d
# and 4d atomic orbitals.  Density matrix embedding theory (DMET) provides a
# method to generate CASSCF initial guess in terms of localized orbitals.
# Given DMET impurity and truncated bath, we can select Fe 3d and 4d orbitals
# and a few entangled bath as the active space.
#


##################################################
#
# Quintet
#
##################################################
d=1.5
mol = gto.Mole()
mol.atom = [
    ['O',(d+0.504284,0.0,0.758602)],
    ['H',(d,0.0,0.0),],
    ['H',(d+2*0.504284,0.0,0.0)],
    ['Mg',(0.0, 0.0, 0.0)]
]
mol.charge=2
mol.spin=0
mol.basis = 'sto3g'
mol.verbose = 4
mol.output = 'fepor.out'
mol.spin = 4
mol.symmetry = True
mol.build()

mf = scf.ROHF(mol)
mf = scf.fast_newton(mf)
# ao_labels are keyword(s) to match the AOs generated by mol.ao_labels()
# function.  ao_labels can be a string, or a pattern of regular expression
# or a list of strings/patterns.  The code can detect the format and find
# the AOs to be taken into the active space.
# See also function gto.mole.search_ao_label for the rules of ao_pattern
ao_pattern = ['Mg 2p', 'O 2p']
ncas, nelecas, mo = dmet_cas.guess_cas(mf, mf.make_rdm1(), ao_pattern)
mc = mcscf.approx_hessian(mcscf.CASSCF(mf, ncas, nelecas))
mc.kernel(mo)
e_q = mc.e_tot  # -2244.82910509839

print('E(T) = %.15g  E(Q) = %.15g  gap = %.15g' % (e_t, e_q, e_t-e_q))


overwrite output file: fepor.out


NameError: name 'e_t' is not defined

In [13]:
ncas, nelecas,mo

(10,
 14,
 array([[ 2.47155053e-05,  9.96241051e-01, -3.49102359e-04,
          6.56772709e-03,  1.50047617e-03,  0.00000000e+00,
         -2.24284838e-01, -2.71063365e-02,  1.15288440e-01,
          5.30069243e-16, -7.60933297e-02, -6.42358472e-02,
          4.57063235e-02,  1.94444157e-18,  2.58523852e-02,
          3.03279818e-02],
        [-2.26694414e-04,  1.83660211e-02,  6.84068765e-04,
         -1.86010095e-02, -5.37106648e-03,  0.00000000e+00,
          8.53316575e-01,  1.32468295e-01, -5.45888332e-01,
         -2.54895275e-15,  5.55842255e-01,  5.13893712e-01,
         -3.27794193e-01, -1.60536249e-17, -1.64949889e-01,
         -2.89338356e-01],
        [ 3.08251008e-05,  2.82156729e-06,  5.19053153e-04,
          4.43864502e-03,  1.20896188e-03,  5.20867677e-20,
         -1.78652039e-02, -8.07605693e-01, -2.12903775e-01,
         -5.64037533e-16, -3.46788550e-01, -7.44407630e-03,
         -6.59030610e-01,  7.89038464e-19,  9.11492367e-02,
          4.75995542e-02],
        [

In [47]:
d=1.5
mol = gto.Mole()
mol.atom = [
    ['O',(d+0.504284,0.0,0.758602)],
    ['H',(d,0.0,0.0),],
    ['H',(d+2*0.504284,0.0,0.0)],
    ['Mg',(0.0, 0.0, 0.0)]
]
mol.charge=2
mol.basis = 'sto3g'
mol.spin = 0
mol.build()

mf = dft.RKS(mol).x2c()
mf.kernel()

#
# The active space can be generated by pyscf.mcscf.avas.avas function
#
# See also 43-dmet_cas.py and function gto.mole.search_ao_label for the rules
# of "ao_labels" in the following
ao_labels = ['Mg 2p', 'O 2p','H 2s']
norb, ne_act, orbs = avas.avas(mf, ao_labels, canonicalize=False)

weights = numpy.ones(13)/13
#solver1 = fci.addons.fix_spin(fci.direct_spin1.FCI(mol), ss=2) # <S^2> = 2 for Triplet
#solver1.spin = 2
#solver1.nroots = 6
#solver2 = fci.addons.fix_spin(fci.direct_spin1.FCI(mol), ss=0) # <S^2> = 0 for Singlet
#solver2.spin = 0
#solver2.nroots = 7

#mycas = mcscf.CASSCF(mf, norb, ne_act)
#mycas.chkfile ='fecp2_3dpz.chk'
#mcscf.state_average_mix_(mycas, [solver1, solver2], weights)
#mycas.verbose = 6
#mycas.kernel(orbs)


#
# Parameters like orbital weights can be accessed through AVAS class
#
avas_obj = avas.AVAS(mf, ao_labels)
avas_obj.kernel()
print(avas_obj.occ_weights)
print(avas_obj.vir_weights)
print(avas_obj.ncas)
print(avas_obj.nelecas)
print(avas_obj.mo_coeff.shape)

converged SCF energy = -270.842130093854
[-1.98693787e-16 -9.66581511e-17  1.70534284e-16  1.86061819e-16
  7.71005053e-01  8.96644610e-01  9.69819683e-01  9.94498816e-01
  9.94520238e-01  9.94761777e-01]
[ 2.12202549e-01 -1.81386222e-17  4.66093028e-04  5.30447683e-04
  6.80697545e-03  8.59773309e-02]
7
12
(16, 16)


In [44]:
weights=numpy.append(avas_obj.occ_weights,avas_obj.vir_weights)
weights[weights<0.1]=0
orbs=numpy.nonzero(weights)

In [46]:
orbs[0]

array([ 4,  5,  6,  7,  8,  9, 10])

In [21]:
#!/usr/bin/env python
#
# Author: Qiming Sun <osirpt.sun@gmail.com>
#

import sys
from pyscf import gto, scf
from pyscf import lo
from pyscf.tools import molden

'''
Write orbitals in molden format
'''

mol = gto.M(
    atom = '''
  O  '''+str(d+0.504284)+''' 0.0 0.758602
  H  '''+str(d)+''' 0.0 0.0
  H  '''+str(d+2*0.504284)+''' 0.0 0.0
  Mg  0.0, 0.0, 0.0
           ''',
    basis = 'sto3g',
    charge=2,
    symmetry = 1)

mf = scf.RHF(mol)
mf.kernel()

#
# First method is to explicit call the functions provided by molden.py
#
with open('C6H6mo.molden', 'w') as f1:
    molden.header(mol, f1)
    molden.orbital_coeff(mol, f1, mf.mo_coeff, ene=mf.mo_energy, occ=mf.mo_occ)

#
# Second method is to simply call from_mo function to write the orbitals
#
c_loc_orth = lo.orth.orth_ao(mol)
molden.from_mo(mol, 'C6H6loc.molden', c_loc_orth)


#
# Molden format does not support high angular momentum basis.  To handle the
# orbitals which have l>=5 functions, a hacky way is to call molden.remove_high_l
# function.  However, the resultant orbitals may not be orthnormal.
#
mol = gto.M(
    atom = 'He 0 0 0',
    basis = {'He': gto.expand_etbs(((0, 3, 1., 2.), (5, 2, 1., 2.)))})
mf = scf.RHF(mol).run()
try:
    molden.from_mo(mol, 'He_without_h.molden', mf.mo_coeff)
except RuntimeError:
    print('    Found l=5 in basis.')
    molden.from_mo(mol, 'He_without_h.molden', mf.mo_coeff, ignore_h=True)

converged SCF energy = -271.391361739475
converged SCF energy = -2.61156726939333
